# Travaux pratiques 3

Ce dernier TP est décomposé en deux parties. Dans la première partie, vous allez expliquer la variable indiquant le maximum d'ozone en fonction des températures. Dans la seconde partie, vous allez prédire le maximum d'ozone pour une nouvelle ville (en fonction de ces températures). Dans le premier cas, l'objectif est d'estimer des paramètres d'intérêt et vous avez accès au maximum d'ozone. Dans le second cas, l'objectif est de prédire le maximum d'ozone.

# Chargement des librairies

In [ ]:
library(missMDA)  # Inclut le dataset ozone
library(VIM)  # Pour la visualisation de valeurs manquantes
library(mice)  # Pour l'imputation multiple
library(misaem)  # Pour le modèle linéaire avec valeurs manquantes
library(parameters)  # Pour agréger les coefficients de l'imputation multiple
library(caret)  # La boîte à outils de machine learning
library(e1071)  # Pour les SVM
library(xgboost)  # Pour le gradient boosting

# Chargement des données

Dans tout ce TP, vous allez considérez les 4 premières variables du jeu de données `ozone` que vous avez découvert au TP1.

La variable d'intérêt est le maximum d'ozone (`maxO3`) et les variables descriptives sont les températures à 9h, 12h et 15h (`T9`, `T12` et `T15`).

In [ ]:
data(ozone) # chargement des données
ozone <- ozone[, 1:4] # sélection des 4 premières variables (colonnes)

In [ ]:
head(ozone)

Comme vous le voyez ci-dessus, la variable `maxO3` contient des valeurs manquantes. C'est un cas en dehors du domaine de compétences développées dans ce MOOC. Dans la suite, nous ne considérons que les villes pour lesquelles le maximum d'ozone est observé.

In [ ]:
ozone <- ozone[, 1:4]
ozone <- ozone[-which(is.na(ozone$maxO3)), ]

On vérifie qu'il n'y a plus aucune valeur manquante pour `maxO3` en utilisant la librairie `VIM` et le code suivant.

In [ ]:
a <- aggr(ozone, plot = FALSE)
par(bg = "white") # définit le fond en blanc
plot(a, numbers = TRUE, prop = FALSE)

# Estimation


Le but est de trouver les paramètres $\beta_0$, $\beta_1$, $\beta_2$ et $\beta_3$, qui expliquent la variable `maxO3` en fonction des variables de température `T9`, `T12` et `T15`, c'est-à-dire tels que:
$$\mathrm{maxO3}=\beta_0+\beta_1\mathrm{T9}+\beta_2\mathrm{T12}+\beta_3\mathrm{T15}+\textrm{bruit gaussien},$$
où le $\textrm{bruit gaussien}$ est un terme modélisant le bruit contenu dans les données, c'est une variable aléatoire qui suit une distribution Gaussienne.

Vous allez voir quatre méthodes qui permettent de gérer les valeurs manquantes :
* la suppression des lignes incomplètes,
* l'algorithme EM,
* l'imputation par la moyenne,
* imputation multiple (question 4).

## Question 1 : suppression des lignes incomplètes

Le code suivant permet d'estimer les paramètres, en supprimant les lignes qui contiennent des valeurs manquantes (lignes incomplètes).

In [ ]:
regression.complete.cases = lm(maxO3 ~ T9 + T12 + T15, data = ozone)
regression.complete.cases$na.action # affichage des indices des lignes supprimées

Les valeurs estimées des paramètres de la régression linéaire sont affichées ci-dessous. L'estimation pour `(Intercept)` correspond à $\beta_0$.

In [ ]:
coef(regression.complete.cases)

On peut afficher les intervalles de confiance à 95% pour les coefficients estimés.

In [ ]:
confidence.complete.cases <- confint(regression.complete.cases)
confidence.complete.cases

Pourquoi cette stratégie n'est-elle pas adaptée ?

### Solution

Si les lignes incomplètes sont supprimées, il y a beaucoup d'information perdue. Au total, on garde 65 points sur 99 (presque 1/3 de supprimés). Le nombre de lignes complètes peut se lire sur le graphique de droite dans le chargement des données.

In [ ]:
dim(ozone)[1] # nombre de lignes total

## Question 2 : algorithme EM

Le code suivant utilise la librairie `misaem` qui permet d'estimer les coefficients $\beta_0$, $\beta_1$, $\beta_2$ et $\beta_3$ de la régression linéaire en utilisant l'algorithme Espérance-Maximisation (EM), sans supprimer les lignes incomplètes.

In [ ]:
regression.em <- miss.lm(maxO3 ~ T9 + T12 + T15, data = ozone)

Les valeurs estimées des coefficients de la régression linéaire sont affichées ci-dessous.

In [ ]:
coef(regression.em)

Est-ce que les résultats sont semblables à ceux obtenus en question 1? Commentez.

### Solution

Les coefficients ont le même signe que lorsqu'ils sont estimés en supprimant les lignes incomplètes. Ils n'ont par contre pas les mêmes valeurs. Les coefficients obtenus précédemment sont sûrement biaisés, les lignes contenant des valeurs manquantes ayant été supprimées.


## Question 3 : imputation par la moyenne

Comparez ces résultats à une autre stratégie pour l'estimation en présence de valeurs manquantes, qui consiste à imputer par la moyenne puis appliquer une régression linéaire sur le jeu de données imputé.

In [ ]:
impute.mean <- function(data) { #fonction d'imputation par la moyenne
  data.imputed <- data
  for (j in 1:dim(data)[2]) {
    data.imputed[which(is.na(data[, j])), j] <- mean(data[, j], na.rm = TRUE)
  }
  return(data.imputed)
}

In [ ]:
ozone.mean <- impute.mean(ozone)

In [ ]:
regression.imp.mean <- lm(maxO3 ~ T9 + T12 + T15, data = ozone.mean)
coef(regression.imp.mean)

### Solution

Pour le coefficient de la variable `T9`, le signe n'est pas le même. L'interprétation des résultats est donc assez différente. Dans le module et TP1, on a vu que l'imputation par la moyenne déforme la distribution du jeu de données. La stratégie d'imputation par la moyenne puis régression donne des résultats biaisés.

## Question 4 : imputation multiple

Dans l'estimation des coefficients, l'incertitude due à la présence de valeurs manquantes n'est pas reflétée. Vous allez maintenant utiliser la librairie `mice` qui permet d'obtenir plusieurs jeux de données imputés, sur lesquels une régression linéaire est ensuite appliquée. Cette méthode permet d'obtenir des intervalles de confiance pour les coefficients estimés. Elle est appelée imputation multiple, même si c'est en fait aussi une méthode d'estimation ! Ici, contrairement à la librairie `misaem`, il y a une étape d'imputation et une étape d'estimation des coefficients (régression linéaire sur les jeux de données imputés).  

La ligne de code suivante permet d'obtenir $m$ jeux de données imputés.

In [ ]:
mult.imp <- mice(ozone, m = 20, print = FALSE)

On applique ensuite la régression linéaire sur tous les jeux de données imputés.

In [ ]:
fit <- with(mult.imp, lm(maxO3 ~ T9 + T12 + T15))

On peut afficher les deux premiers jeux de données imputés, et leurs régressions linéaires.

In [ ]:
complete.dataset.1 <- complete(mult.imp, 1) # jeu de données imputé 1
head(complete.dataset.1, 10)

coef(fit$analyses[[1]]) # analyse du jeu de donnée imputé 1

In [ ]:
complete.dataset.2 <- complete(mult.imp, 2) # jeu de données imputé 2
head(complete.dataset.2, 10)

coef(fit$analyses[[2]]) # analyse du jeu de données imputé 2

### Question 4a

Que remarquez-vous?

### Solution

Les deux premiers jeux de données imputés obtenus avec `mice` ont bien des valeurs prédites différentes pour les valeurs manquantes, et les estimations des coefficients sont différentes.

Un exemple de valeur prédite différente est donné ci-dessous.

In [ ]:
paste("Valeur prédite dans le jeu de données imputé 1:", complete.dataset.1[which(is.na(ozone$T9[1:10])), 2])

paste("Valeur prédite dans le jeu de données imputé 2:", complete.dataset.2[which(is.na(ozone$T9[1:10])), 2])


### Question 4b

Le code suivant permet d'obtenir les coefficients estimés aggrégés sur les $m=20$ jeux de données imputés.

In [ ]:
res.mult <- model_parameters(fit)

res.mult$Coefficient

Pour obtenir des intervalles de confiance à 95% pour chaque coefficient, on applique le code suivant.

In [ ]:
confidence.mult <- data.frame(as.numeric(res.mult$CI_low),
                             as.numeric(res.mult$CI_high),
                             row.names = res.mult$Parameter)
colnames(confidence.mult) <- c("2.5%", "97.5%")

confidence.mult

Comparez les résultats avec ceux obtenus précédemment, notamment ceux de la question 1 en supprimant les lignes incomplètes, pour les intervalles de confiance.

### Solution

Les valeurs des coefficients sont différentes, mais restent de même signe que ceux obtenus en question 1 (suppression des lignes incomplètes) et question 2 (algorithme EM).
Pour les intervalles de confiance, on peut remarquer que leur longueur est plus grande lorsque les lignes incomplètes sont supprimées.

In [ ]:
# Longueur des intervalles de confiance lorsque les lignes incomplètes sont supprimées
length.confidence.complete.cases <- confidence.complete.cases[ ,2]-confidence.complete.cases[ ,1]
length.confidence.complete.cases

In [ ]:
# Longueur des intervalles de confiance lorsque mice est utilisée
length.confidence.mult <- confidence.mult[ ,2]-confidence.mult[ ,1]
length.confidence.mult

# Exercice 2 : prédiction

Dans la première partie, nous avons vu comment estimer les paramètres d'une régression linéaire, modélisant la variable d'intérêt $y=$ `maxO3` en fonction des variables explicatives $X=$ `[T9, T12, T15]`, même si celles-ci avaient des valeurs manquantes. Rappelons le modèle que nous avons utilisé :

In [ ]:
summary(regression.em)

Nous nous sommes appuyés sur un jeu de données d'entraînement $(y_{train},~X_{train})$, construit lors d'une étude réalisée de juin à septembre 2001. Affichons à nouveau ses premières lignes :

In [ ]:
print(head(ozone, 15))

## Question 1

Imaginons maintenant que nous recevions un nouveau cas $X_{new}$ où $y$ est inconnue, et que nous souhaitions déterminer sa valeur la plus probable.

Concrètement, on se place à une date ultérieure, le 1/07/2025, on mesure une température de 15.1°C à 9h, 16.8°C à 12h, et 19.7°C à 15h, et on souhaite déterminer la valeur maximale d'ozone sur la journée à partir de ces seules températures.

C'est ce qu'on appelle la **prédiction**.

In [ ]:
X.new <- data.frame(T9 = 15.1, T12 = 16.8, T15 = 19.7, row.names = '20250701')
print(X.new)
print(predict(regression.em, X.new))

Pourquoi rencontrerait-on une telle situation en pratique, avec le jeu de données `ozone` ? Donnez un exemple de scénario.

Ensuite, dans votre domaine, trouvez un autre exemple de scénario impliquant de la prédiction.

### Solution

Dans notre cas du jeu de données `ozone`, on peut imaginer que la variable d'intérêt `maxO3` est technique ou chère à mesurer et qu'on n'en avait la possibilité que dans le cadre de l'étude de 2001, et on aimerait donc pouvoir la prédire au quotidien à partir des mesures de température uniquement, puisqu'elles sont faciles à réaliser.

## Question 2



Une autre difficulté s'ajoute dans notre cas. En général, si les nouvelles données sont récoltées dans les mêmes conditions que le jeu d'entraînement, il y a de fortes chances qu'il y ait aussi des valeurs manquantes dans $X_{new}$. Il faut donc le prendre en compte.

In [ ]:
X.new.na <- data.frame(T9 = 15.1, T12 = NA, T15 = 19.7, row.names = '20250701')
print(X.new.na)

La première approche "naïve" est d'essayer de prédire directement avec le modèle qu'on a entraîné sur la nouvelle donnée...

In [ ]:
predict(regression.em, X.new.na)

Cela fonctionne-t-il ? Est-ce surprenant ?

### Solution

Cela fonctionne ! Alors pourquoi aller plus loin ?

En réalité cette méthode du package `misaem` est loin d'être naïve, elle cache un algorithme complexe, qui examine toutes les combinaisons possibles de variables dans le modèle (et repose sur des hypothèses assez restrictives). Quand on a beaucoup de variables et beaucoup de prédictions à réaliser, le temps de calcul peut exploser. De plus, elle suppose que les valeurs manquantes n'ont pas d'influence particulière sur la réponse $y$, ce qui peut être faux.

## Question 3 : approche en deux étapes avec l'imputation

Une alternative simple qui fonctionne avec n'importe quel modèle de machine learning, et qui est très souvent remarquablement efficace en prédiction, est **l'imputation par la moyenne** (ou la médiane, ça n'a pas d'importance dans ce cas).

Implémentons-la à l'aide de la fonction `preProcess` du package `caret`, la boîte à outils de machine learning (qui ne supporte que la médiane).

**Note importante :** pour utiliser cette méthode, il faut calculer les médianes **sur le jeu d'entraînement**, et imputer à la fois le jeu d'entraînement et les nouvelles données par ces mêmes médianes.

Dans la cellule suivante, commentez chaque ligne en expliquant ce qu'elle fait.

In [ ]:
preproc <- preProcess(ozone[c("T9", "T12", "T15")], method = "medianImpute")  # ???
print(preproc$median)  # ???

ozone.imputed <- predict(preproc, newdata = ozone)  # ???
print(head(ozone.imputed))  # ???

model <- miss.lm(maxO3 ~ ., data = ozone.imputed)  # ???
print(summary(model))  # ???

X.new.imputed <- predict(preproc, newdata = X.new.na)  # ???
print(X.new.imputed)  # ???

predictions <- predict(model, newdata = X.new.imputed)  # ???
print(predictions)  # ???

### Solution

In [ ]:
preproc <- preProcess(ozone[c("T9", "T12", "T15")], method = "medianImpute")  # Calcul des médianes des colonnes du jeu d'entraînement
print(preproc$median)  # Affichage des médianes calculées

ozone.imputed <- predict(preproc, newdata = ozone)  # Imputation du jeu d'entraînement avec les médianes précalculées
print(head(ozone.imputed))  # Affichage des premières lignes du jeu de données imputé

model <- miss.lm(maxO3 ~ ., data = ozone.imputed)  # Entraînement du modèle linéaire sur le jeu imputé
print(summary(model))  # Affichage des paramètres du modèle

X.new.imputed <- predict(preproc, newdata = X.new.na)  # Imputation des nouvelles données par les médianes précalculées
print(X.new.imputed)  # Affichage des nouvelles données imputées

prediction <- predict(model, newdata = X.new.imputed)  # Prédiction sur les nouvelles données imputées
print(prediction)  # Affichage de la prédiction

## Question 4

Le modèle linéaire a l'avantage de permettre d'interpréter complètement la relation entre $y$ et $X$, mais quelles sont ses limites ? Connaissez-vous d'autres modèles de *machine learning* et leurs avantages ?

### Solution

Le modèle linéaire n'est pas toujours le plus juste, car il ne modélise que des effets linéaires. En fait il est possible d'ajouter des effets combinés ou non linéaires, mais il faut les renseigner à la main, le modèle ne peut pas les deviner automatiquement. Par exemple, dans le modèle suivant :

$$\mathrm{maxO3}=\beta_0+\beta_1\times\mathrm{T9}+\beta_2\times\mathrm{T12}+\beta_3\times\mathrm{T15} +\beta_4\times\left(T12-T9\right)^2 +\textrm{bruit gaussien},$$

on a ajouté à la main un effet quadratique de la variation de température entre 9h et 12h, mais ça reste un modèle linéaire avec cinq paramètres, $\beta_0, \beta_1, \beta_2, \beta_3, \beta_4$.


Pour capturer automatiquement des effets combinés ou non linéaires, d'autres modèles existent, comme la machine à vecteurs de support (*Support Vector Machine, SVM*) ou les modèles à base d'arbres de décision comme le gradient boosting.

## Question 5

L'avantage de l'imputation est qu'elle fonctionne avec n'importe quel modèle de machine learning. Reproduisez l'approche de la question 3 en remplaçant le modèle linéaire par un *support vector machine* (SVM) :

In [ ]:
# model <- svm(maxO3 ~ ., data = ...)

### Solution

In [ ]:
preproc <- preProcess(ozone[c("T9", "T12", "T15")], method = "medianImpute")  # Calcul des médianes des colonnes du jeu d'entraînement
print(preproc$median)  # Affichage des médianes calculées

ozone.imputed <- predict(preproc, newdata = ozone)  # Imputation du jeu d'entraînement avec les médianes précalculées
print(head(ozone.imputed))  # Affichage des premières lignes du jeu de données imputé

model <- svm(maxO3 ~ ., data = ozone.imputed)  # Entraînement du SVM sur le jeu imputé
print(summary(model))  # Affichage des paramètres du modèle

X.new.imputed <- predict(preproc, newdata = X.new.na)  # Imputation des nouvelles données par les médianes précalculées
print(X.new.imputed)  # Affichage des nouvelles données imputées

prediction <- predict(model, newdata = X.new.imputed)  # Prédiction sur les nouvelles données imputées
print(prediction)  # Affichage de la prédiction

## Question 6 : gradient boosting

En dehors du modèle linéaire, il existe une autre classe de modèles qui incorporent les valeurs manquantes de manière naturelle : les modèles à base d'arbres de décisions. Parmi eux, on peut citer :

* **l'arbre de décision simple**, un modèle bien interprétable mais difficile à paramétrer et peu efficace en prédiction ;
* **la forêt aléatoire**, un ensemble d'arbres de décision choisis aléatoirement, qui est un modèle robuste, facile à paramétrer, et très efficace en prédiction ;
* **le boosting**, une séquence d'arbres s'affinant au fil de l'entraînement, qui est imbattable en prédiction, mais requiert plus d'expérience dans sa paramétrisation.

Nous nous concentrerons ici sur le **boosting** avec la librairie `xgboost`, qui est très utilisée en pratique et est une des seules librairies à implémenter cette gestion automatique des données manquantes en R.

Exécutez la cellule suivante avec chacune des 7 versions proposées de xgboost, en décommentant la ligne. Déterminez la particularité de chaque version, et son utilité. Vous pouvez vous aider de la documentation de xgboost : https://cran.r-project.org/web/packages/xgboost/xgboost.pdf#page=58

Comparez également les résultats. Quelles prédictions vous semblent les plus pertinentes ?

In [ ]:
### covariates <- as.matrix(ozone[, c("T9", "T12", "T15")])
label <- ozone$maxO3

# 1.
# xgb.model <- xgboost(data = covariates, label = label, nrounds = 30, missing = NA, booster = 'gbtree', objective = "reg:squarederror")
# 2.
# xgb.model <- xgboost(data = covariates, label = label, nrounds = 60, missing = NA, booster = 'gbtree', objective = "reg:squarederror")
# 3.
# xgb.model <- xgboost(data = covariates, label = label, nrounds = 300, missing = NA, booster = 'gbtree', objective = "reg:squarederror", early_stopping_rounds = 5)
# 4.
# xgb.model <- xgboost(data = covariates, label = label, nrounds = 30, missing = 0, booster = 'gbtree', objective = "reg:squarederror")
# 5.
# xgb.model <- xgboost(data = covariates, label = label, nrounds = 30, missing = NA, booster = 'gblinear', objective = "reg:squarederror")
# 6.
# xgb.model <- xgboost(data = covariates, label = label, nrounds = 30, missing = NA, booster = 'gbtree', objective = "reg:squaredlogerror")
# 7.
# xgb.model <- xgboost(data = covariates, label = label, nrounds = 30, missing = NA, booster = 'gbtree', objective = "reg:squarederror", verbose = FALSE)


prediction.xgb <- predict(xgb.model, as.matrix(X.new.na))
print(prediction.xgb)

### Solution

1. La première version utilise les paramètres par défaut.
2. `nrounds = 60` : on utilise une séquence de 60 arbres plutôt que 30, ce qui est censé améliorer la performance, mais prend deux fois plus de temps.
3. `nrounds = 300, early_stopping_rounds = 5` : la méthode d'early stopping est très efficace pour avoir le modèle le plus performant possible sans perdre de temps. A chaque tour, on fait un test : si la performance ne s'est pas améliorée depuis 5 tours, alors on arrête. Par conséquent, on peut mettre un nombre de tours très grand, ce n'est pas grave, on s'arrêtera sûrement avant !
4. `missing = 0` : on déclare ainsi que les valeurs manquantes sont encodées par des 0 (ou une autre valeur, -1 par exemple ou 9999) au lieu de `NA`. Selon le jeu de données qu'on utilise, ce paramètre peut être très utile !
5. `booster = 'gblinear'` : on utilise comme modèle de base un modèle linéaire plutôt qu'un arbre de décision. A tester car ça peut être plus performant dans certains cas, notamment quand on a beaucoup de variables ou qu'on suspecte des effets linéaires.
6. `objective = 'reg:squaredlogerror'` : on peut également changer le critère d'erreur, il y en a plusieurs possible, et il faut choisir celui qui est le plus adapté à son problème. Utiliser la *Root Mean Squared **Log** Error* (RMS**L**E), squared**log**error dans xgboost) au lieu de la plus classique *Root Mean Squared Error* (RMSE, squarederror dans xgboost) a deux intérêt : limiter l'impact des valeurs aberrantes qui ont des erreurs énormes, et pénaliser davantage les sous-estimations que les sur-estimations.
7. `verbose = FALSE` : n'affiche pas les performances intermédiaires, pour plus de légèreté.

L'objectif `'reg:squaredlogerror'` n'est clairement pas adapté ici, donnant une prédiction aberrante. Pour les autres, il est difficile d'évaluer la qualité de la prédiction puisqu'on ne connaît pas la vraie valeur, d'où l'intérêt de tester le modèle sur des "nouvelles" valeurs qui ne font pas partie du jeu d'entraînement, mais dont on connaît la réponse. Cette démarche de validation de modèle n'est pas détaillée ici, mais elle est essentielle dans un processus de machine learning.

## Question 7 : et si $X_{new}$ est toujours complet ?

Est-il possible qu'il y ait des valeurs manquantes dans le jeu d'entraînement, mais jamais dans les nouvelles données ? Donnez un exemple de scénario, dans le cas des données `ozone`, puis dans votre domaine d'étude.

Que faire dans ce cas ?

In [ ]:
X.new <- data.frame(T9 = 15.1, T12 = 17.2, T15 = 19.7, row.names = '20250701')
print(X.new)

### Solution

On peut tout à fait concevoir que les nouvelles valeurs de température n'aient jamais de valeurs manquantes. Cela peut être parce que les capteurs de température ont été changés après l'étude pour des versions plus robustes, ou parce que la connexion internet a été améliorée, par exemple.

Bien sûr, tout ce qu'on a vu avec $X_{new}$ incomplet peut s'appliquer dans ce cas plus facile, et heureusement (on n'est jamais obligé d'avoir des valeurs manquantes !).

Cependant, si on est certain qu'il n'y aura plus jamais de valeurs manquantes, alors il vaut mieux ne pas s'entraîner avec des valeurs manquantes pour ne pas introduire de biais. En particulier, les méthodes à base d'arbres de décision risquent de trop s'appuyer sur la présence de `NA`, et donner de fausses prédictions. Il est donc recommandé de faire une **imputation** avant d'entraîner le modèle.

## Conclusion pour la prédiction

Retenons trois informations importantes:
* La seule manière de gérer les valeurs manquantes en entraînement et en prédiction pour n'importe quel modèle de machine learning est de faire d'abord une **imputation**.
* Imputer le jeu d'entraînement est aussi la méthode la plus souvent adaptée si on sait qu'on n'aura pas de données manquantes en prédiction.
* Au cas par cas, certains modèles ont des implémentations qui permettent de gérer nativement les valeurs manquantes sans faire d'imputation, et d'optimiser ainsi les performances. C'est le cas du modèle linéaire, et des modèles à base d'arbres de décision comme le gradient boosting.